## Introducción

En el siguiente proyecto trabajaré con la base de datos de una compañía móvil en la cual analizaré el comportamiento de sus clientes con el objetivo de recomendar un nuevo plan. Para realizar estas recomendaciones, desarrollaré un modelo de clasificación que escoja correctamente cual de los planes se ajusta mejor a cada usuario de acuerdo con sus características. 